In [8]:
Packages <- c("dplyr",  "nleqslv", "broom","cubature", "geosphere", "data.table",  "ggplot2", "bbmle", "stringr",  "lubridate", "RColorBrewer")

invisible(suppressPackageStartupMessages(lapply(Packages, library, character.only = TRUE)))

setwd('/local/home/katrinac/oceanography')
"%!in%" <- function(x,table) match(x,table, nomatch = 0) == 0
source("~/parentage/kernel_fitting/1340_loci/functions/ll_kt_both_bbmle.R")
#source("~/oceanography/scripts/PredictedProportions.R")

#read in the kernel fitting summary
kernels <- fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/kernel_fitting_summary.csv")
kernel2012_14 <- fread(file="~/oceanography/empirical_data/genetics/GenKernelsForROMSComp2012-14.csv")

#read in the centroids adjusted for the simulation, so the Magbangons combined 
#centroids <- fread(file="~/oceanography/script_output/SurveyData/SimulationCentroids.csv")
Centroids <- fread(file="~/oceanography/empirical_data/site_centroids_SimTest.csv")
setorder(centroids, site)
#read in the table with number of recruits sampled at each site for each year
AnnualRecsSamp <- fread(file="~/oceanography/script_output/SurveyData/AnnualRecruitsSampled.csv")
#read in the table of the proportion of anemones sampled at each site for each year
PropSamp <- fread(file="~/oceanography/script_output/SurveyData/ProportionHabitatSampled.csv")

#read in the ROMS simulation connectivity table with metadata, not yet subsetted (*but check this)
DateJoin <- fread(file="~/oceanography/script_output/ROMSDataTables/LongFormConnWithProbs.csv")
setnames(DateJoin, c("YearSampled", "SurvWeight", "Dest", "DestNOffsAnnual", "DestPropSamp", "DestTotalAnems", "Source", "SourceNOffsAnnual", "SourcePropSamp", "SourceTotalAnems", "SimMonth", "SimDay", "SimYear", "SimMonsoon", "DailyParticles", "ParticleID", "SourceSize"), 
        c("year_sampled", "surv_weight", "dest", "dest_n_offs_annual", "dest_prop_samp", "dest_total_anems", "source", "source_n_offs_annual", "source_prop_samp", "source_total_anems", "sim_month", "sim_day", "sim_year", "sim_monsoon", "daily_particles", "particle_id", "source_size"))

#make vectors defining sites we didn't sample, but that are in the model, and the sandflats specifically 
UnsampledSites <- c("SF1", "SF2", "SF3", "SF4", "SF5", "SF6", "Pangasugan", "Other", "CAI") 
SandFlats <- c("SF1", "SF2", "SF3", "SF4", "SF5", "SF6") 

#make the constant inputs for the kernel fitting function
#distance matrix using the centroids with combined Magbangon
### List of source locations
sites_source <- centroids

### List of destination locations
sites_dest <- centroids

dist_mat_m <- distm(sites_source[,c('lon','lat')], sites_source[,c('lon','lat')], fun=distVincentyEllipsoid)
Distances <- dist_mat_m*10^-3
#read in the reef areas for the kernel fitting
area <- fread("~/oceanography/empirical_data/site_area_header_nonsurveyed_simulation_kernels_test.csv") %>%
    arrange(site) %>%
    filter(site %!in% c("near_north_full1", "near_north_full2", "near_north_full3", "near_south_full1", "near_south_full2", "near_south_full3")) %>%
    mutate(kmsq=msq*10^-6)# %>%
    #select(kmsq) #need to uncomment for functions to work
setorder(area, site)
reef_sizes <- as.matrix(Area$kmsq)

#make a table with the survey information for each site (how many fish sampled, prop anems sampled, total number of anems at site)
SurveyData <- AnnualRecsSamp[PropSamp, on=.(year=end_year, site)][#join the sampling tables together
    is.na(n_offs_gen), n_offs_gen := 0][#change NA's to 0
    ,-"time_frame"]#drop the time_frame column, we can key with end_year
setnames(SurveyData, c("PropAnemSamp", "TotalAnems") c("prop_anem_samp", "total_anems"))
#setkey(SurveyData, site)
#check all sites are represented in centroids and area (and indirectly distances, which comes from centroids)
#Area[site %!in% centroids$site] #should be nothing

#genetic data to make predicted proportions
Assignments2012_4 <- as.matrix(read.csv("~/parentage/kernel_fitting/1340_loci/input/parentage_matrix_2012-14ForROMSComp.csv", header=FALSE))
Adult_sample_proportions2012_4 <- as.matrix(read.csv("~/parentage/kernel_fitting/1340_loci/input/prop_samp12-14.csv", header=FALSE))
Sampled_reefs2012_4 <- as.matrix(read.csv("~/parentage/kernel_fitting/1340_loci/input/site_index12-14.csv", header=FALSE))

Distances_gen <- as.matrix(read.csv("~/parentage/kernel_fitting/1340_loci/input/distance_matrix_unsurveyed.csv", header=FALSE))
Reef_sizes_gen <- as.matrix(read.csv("~/parentage/kernel_fitting/894_loci/area_unsurveyed.csv", header=FALSE))
Centroids_gen <- as.matrix(read.csv("~/parentage/kernel_fitting/894_loci/centroids_unsurveyed.csv", header=T))

Assignments2012 <- as.matrix(read.csv("~/parentage/kernel_fitting/1340_loci/input/parentage_matrix12_corrected.csv", header=FALSE))
Adult_sample_proportions2012 <- as.matrix(read.csv("~/parentage/kernel_fitting/1340_loci/input/prop_samp12.csv", header=FALSE))
Sampled_reefs2012 <- as.matrix(read.csv("~/parentage/kernel_fitting/1340_loci/input/site_index12.csv", header=FALSE))

Assignments2013 <- as.matrix(read.csv("~/parentage/kernel_fitting/1340_loci/input/parentage_matrix13_corrected.csv", header=FALSE))
Adult_sample_proportions2013 <- as.matrix(read.csv("~/parentage/kernel_fitting/1340_loci/input/prop_samp13.csv", header=FALSE))
Sampled_reefs2013 <- as.matrix(read.csv("~/parentage/kernel_fitting/1340_loci/input/site_index13.csv", header=FALSE))

Assignments2014 <- as.matrix(read.csv("~/parentage/kernel_fitting/1340_loci/input/parentage_matrix14_corrected.csv", header=FALSE))
Adult_sample_proportions2014 <- as.matrix(read.csv("~/parentage/kernel_fitting/1340_loci/input/prop_samp14.csv", header=FALSE))
Sampled_reefs2014 <- as.matrix(read.csv("~/parentage/kernel_fitting/1340_loci/input/site_index14.csv", header=FALSE))

#Allison's abundance time series data 
download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Script_outputs/females_df_F.RData?raw=true", destfile = "~/oceanography/empirical_data/genetics/females_df_F.RData")
load("~/oceanography/empirical_data/genetics/females_df_F.RData")
Abundance <- as.data.table(females_df_F)

In [20]:
SurveyData <- AnnualRecsSamp[PropSamp, on=.(year=end_year, site)][#join the sampling tables together
    is.na(n_offs_gen), n_offs_gen := 0][#change NA's to 0
    ,-"time_frame"]#drop the time_frame

SurveyData <- Abundance[, c("year", "site", "nF")][SurveyData, on=.(year, site)]#join in Allison's estimate of female abundance. There are NA values, but that's okay we can figure those out when we start thinking about incorporating uncertainty in this


In [21]:
head(SurveyData)

year,site,nF,n_offs_gen,PropAnemSamp,TotalAnems
2012,Cabatoan,8,4,0.4230769,26
2012,Caridad Cemetery,NA,0,0.0000000,4
2012,Caridad Proper,NA,0,0.0000000,4
2012,Elementary School,NA,0,0.0000000,7
2012,Gabas,NA,0,0.0000000,9
2012,Haina,NA,0,0.0000000,104


In [22]:
DateJoin <- fread(file="~/oceanography/script_output/ROMSDataTables/LongFormConnWithProbs.csv")


In [27]:
head(DateJoin)

year_sampled,date,surv_weight,dest,dest_n_offs_annual,dest_prop_samp,dest_total_anems,source,source_n_offs_annual,source_prop_samp,source_total_anems,sim_month,sim_day,sim_year,sim_monsoon,daily_particles,particle_id,source_size
2011,2010-10-01,0.003292181,Palanas,NA,NA,NA,Other,NA,NA,NA,10,10,2010,SWM,9,P1,NA
2011,2010-10-01,0.003292181,Palanas,NA,NA,NA,Other,NA,NA,NA,10,10,2010,SWM,9,P2,NA
2011,2010-10-01,0.003292181,Palanas,NA,NA,NA,Other,NA,NA,NA,10,10,2010,SWM,9,P3,NA
2011,2010-10-01,0.003292181,Palanas,NA,NA,NA,Other,NA,NA,NA,10,10,2010,SWM,9,P4,NA
2011,2010-10-01,0.003292181,Palanas,NA,NA,NA,Other,NA,NA,NA,10,10,2010,SWM,9,P5,NA
2011,2010-10-01,0.003292181,Palanas,NA,NA,NA,Other,NA,NA,NA,10,10,2010,SWM,9,P6,NA


__below is where I left off 06/01/2021. rename columns to be the vector format like_this, and finish making the connectivity matrix for the biophysical simulation next__

In [ ]:
#prep biophysical connectivity matrix
#outside of the loop, trim this to only be the destinations we sampled

dest_sampled <- date_join[dest_prop_samp >0]
dest_sampled <- kernels[Year %in% c("2012", "2013", "2014")][, Year:=as.integer(Year)][,c("Year", "NumParentageMatches")][dest_sampled, on=.(Year=YearSampled)]#add in a column for the observed number of parentage matches
#rename the monsoon column in the full table for consistency
setnames(dest_sampled, "SimMonsoon", "Monsoon")

#each year will require a different set of survey data, so make a list of each and index by site for fast look up
PropSampTable <- SurveyData[PropAnemSamp >0, c("year", "site")]

#make a site index table, use this for Sampled_reefs input in kernel fitting
site_index <- unique(centroids, by="site")[, "site"][, index := .I] #add the row number as the unique site index

#make sure all sampled sites are represented by joining the survey data to the sampled simulation
PropSampTable <- rbind(SurveyData[PropAnemSamp >0 & year %in% c(2012, 2013, 2014), c("year", "site")][, .(Source=site, Dest=site, Year=year)][ #will join to the simulated sampling table by source and dest, so make those each a column from site and preserve the year variable as a key
    , c("Year", "Source", "Dest")][, Monsoon := "NEM"], SurveyData[PropAnemSamp >0 & year %in% c(2012, 2013, 2014), c("year", "site")][, .(Source=site, Dest=site, Year=year)][ #will join to the simulated sampling table by source and dest, so make those each a column from site and preserve the year variable as a key
    , c("Year", "Source", "Dest")][, Monsoon := "SWM"])
unq_survey <- unique(PropSampTable, by=c("Source", "Dest", "Year", "Monsoon"))#add in the diff Monsoon seasons so there are complete parentage matrices later
add_dest <- rbind(SurveyData[year %in% c(2012, 2013, 2014) & PropAnemSamp >0][, c("year", "site")][, Monsoon := "NEM"], 
                  SurveyData[year %in% c(2012, 2013, 2014) & PropAnemSamp >0][ , c("year", "site")][, Monsoon := "SWM"])  #what destinations were sampled, for use with unassigned table




In [18]:
#Biophysical source normalized matrix
#for each source, what is the normalized recruitment at each destination? 
GenSimConn[, annual_source_normalized_recruitment := sum(daily_particles_recruited)/sum(daily_particles_released), by=c("source", "destination","year_sampled")]


year,site,nF,n_offs_gen,PropAnemSamp,TotalAnems
2012,Cabatoan,8,4,0.42307692,26
2012,Caridad Cemetery,NA,0,0.00000000,4
2012,Caridad Proper,NA,0,0.00000000,4
2012,Elementary School,NA,0,0.00000000,7
2012,Gabas,NA,0,0.00000000,9
2012,Haina,NA,0,0.00000000,104
2012,Hicgop South,NA,0,0.00000000,18
2012,Magbangon,NA,10,0.45938375,139
2012,Palanas,49,13,0.28985507,138
2012,Poroc Rose,9,9,1.00000000,13


__Code function for likelihood of parentage data given a biophysical model, based on Bode et al. 2019 in Plos Bio__

In [ ]:
#lay out all the pieces
pop_size_vec <- #vector of pop sizes for all reefs (a). This term is also used in parentage kernel fitting, but reef sizes are substituted as a proxy for pop size. This is should be bootstrapped to account for uncertainty.
BioPhysMat <- #source normalized biophysical connectivity matrix. In Eqn. S3.4, this is m ajt/r a (*should it be r at? As in all particles released in time period t?)
prop_samp_vec <- #vector of proportion of habitat sampled for all reefs in time period t
unassigned_vec#from genetic parentage data- a vector of the number of unassigned recruits at each destination reef in the system- we only have this for all sampled reefs.... what should the dimensions be?*


In [ ]:
#format the data

In [ ]:
reef

In [3]:
head(AnnualRecsSamp)

year,site,n_offs_gen
2012,Cabatoan,4
2012,Palanas,13
2012,Poroc Rose,9
2012,Poroc San Flower,9
2012,Sitio Lonas,1
2012,Visca,1


In [ ]:
#Eqn. S3.4 term 1 for loop

for(j in 1:nrow(centroids)){ #for each destination reef in the whole Camotes system
    
dest_unassigned <- sum(unassigned_vec[t==time_period]) #the sum of all unassigned recruits at time period t
    
        for(i in 1:nrow(centroids)){ #term 1- for each source reef in the whole Camotes system, expected unassigned at each destination reef
            
            term1_num <- ((1-prop_samp_vec[i,t== time_period])^2)*pop_size_vec[i]*BioPhysMat[i,j,t]
            
            for(a in 1:nrow(centroids)){ #for all reefs total in the whole Camotes system, denominator of term 1
                
                value_a <-  pop_size_vec[a]*BioPhysMat[a, j, t]
                term1_denom <- sum(value_a)
            }
        
        term1 = term1_num/term1_denom
        }

    for(k in 1:nrow(centroids)){ #term 2- for each source reef in the whole Camotes system, expected assigned at each destination reef
        
        term2_num <- (1-(1-prop_samp_vec[i,t== time_period])^2)*pop_size_vec[i]*BioPhysMat[i,j,t]
            
            for(a in 1:nrow(centroids)){ #for all reefs total in the whole Camotes system, denominator of term 2
                
                value_a <-  pop_size_vec[a]*BioPhysMat[a, j, t]
                term2_denom <- sum(value_a)
            }
        
        term2 = term2_num/term2_denom
        
        }
        
    

prob_unsampled <- dest_unassigned*log(term1)  
prob_sampled <- dest_assigned*log(term2)
ll = prob_unsampled + prob_sampled
}

In [ ]:

for(i in 1:NumSampledReefs){
   This_SS_A = Adult_sample_proportions[i]#same
   for(j in 1:NumSampledReefs){
    SettlersFromAssignedReefs = Settlers[Sampled_reefs[i],Sampled_reefs[j]]#same
    #Not all settlers from assigned reefs will be assigned, because not all adults were sampled
    AssignedSettlers[i,j] = SettlersFromAssignedReefs*(This_SS_A^2 + 2*This_SS_A*(1 - This_SS_A))
    AssignedSettlers[NumSampledReefs+1,j] = AssignedSettlers[NumSampledReefs+1,j] + SettlersFromAssignedReefs*(1-This_SS_A)^2 #The three dots '...' tell matlab that the code on a given line continues on the next line.
   }
}
Unsampled = as.matrix(setdiff(1:NumReefs,Sampled_reefs))


for(j in 1:NumSampledReefs){
   AssignedSettlers[NumSampledReefs+1,j] = AssignedSettlers[NumSampledReefs+1,j] + sum(Settlers[Unsampled,Sampled_reefs[,j]]) 
}
   



__Loop through sampling different proportions of other and CAI source particles and compare the unassigned proportions of total sample particles to the genetic observations from survey data__

In [48]:
PropAssignedTable <- rbind(kernels[Year %in% c("2012", "2013", "2014")][
    , PropAssigned := PercentAssigned/100][ #change to proportion note percent
    , c("Year", "NumParentageMatches", "NumOffsSampled", "PropAssigned")],                      
    unique(kernels[Year %in% c("2012", "2013", "2014")][ #only the years coinciding with the models
    , `:=` (NumParentageMatches=sum(NumParentageMatches), NumOffsSampled=sum(NumOffsSampled), PropAssigned = NumParentageMatches/NumOffsSampled, Year = "2012-4")][ #summarise across the 3 years
    , c("Year","NumParentageMatches", "NumOffsSampled", "PropAssigned")], by="Year"))
    

PropAssignedTable[]

#add in the average sampled proportion of anemones
AvgPropSamp <- SurveyData[PropAnemSamp >0, .(PropAnemSamp = mean(PropAnemSamp)), by="year"][ #average for each site we sampled, how well we sampled
    year %in% c("2012", "2013", "2014")][
    , year :=as.character(year)]

ExpectedPropAssigned <- AvgPropSamp[PropAssignedTable, on=.(year=Year)]
ExpectedPropAssigned$PropAnemSamp[is.na(ExpectedPropAssigned$PropAnemSamp)] <- mean(ExpectedPropAssigned$PropAnemSamp, na.rm = T) #replace the 2012-4 NA with the average from the 3 years

#what's the normalized self recruitment proportion back to the population
ExpectedPropAssigned[, ExpAssigned := NumParentageMatches/(NumOffsSampled*PropAnemSamp)][] #this is the expected assignment for the whole surveyed population if we had sampled all adults (which we kind of do when we use the simulation results)
#at some point... maybe it would be better to compare on a site to site level? idk that's pretty fine scale, I don't know that our ROMS model can be expected to compare so well with that

Year,NumParentageMatches,NumOffsSampled,PropAssigned
2012,3,63,0.04760000
2013,21,150,0.14000000
2014,13,181,0.07180000
2012-4,37,394,0.04761905


year,PropAnemSamp,NumParentageMatches,NumOffsSampled,PropAssigned,ExpAssigned
2012,0.6566669,3,63,0.04760000,0.07251629
2013,0.5793625,21,150,0.14000000,0.24164492
2014,0.5714427,13,181,0.07180000,0.12568749
2012-4,0.6024907,37,394,0.04761905,0.15586735


__For the expected values of recruits from outside of our sampled region, I'm using the intermediate "PredictedProportions" matrix from the Bode kernel fitting script (https://github.com/MikeBode/Parentage_kernel_fitting/blob/master/Kernel_Fitting_Function.m) because it accounts for how well sites were sampled when estimating proportions of recruits from unsampled sites__

In [55]:
#####outside of the loop*****

PropSampTable <- SurveyData[PropAnemSamp >0, c("year", "site")]

#make sure all sampled sites are represented by joining the survey data to the sampled simulation
PropSampTable <- rbind(SurveyData[PropAnemSamp >0 & year %in% c(2012, 2013, 2014), c("year", "site")][, .(Source=site, Dest=site, Year=year)][ #will join to the simulated sampling table by source and dest, so make those each a column from site and preserve the year variable as a key
    , c("Year", "Source", "Dest")][, Monsoon := "NEM"], SurveyData[PropAnemSamp >0 & year %in% c(2012, 2013, 2014), c("year", "site")][, .(Source=site, Dest=site, Year=year)][ #will join to the simulated sampling table by source and dest, so make those each a column from site and preserve the year variable as a key
    , c("Year", "Source", "Dest")][, Monsoon := "SWM"])
#unq_survey <- unique(PropSampTable, by=c("Source", "Dest", "Year", "Monsoon"))#, unique(PropSampTable, by=c("Source", "Dest", "Year"))[, Monsoon := "SWM"]) #add in the diff Monsoon seasons so there are complete parentage matrices later
#add_dest <- rbind(SurveyData[year %in% c(2012, 2013, 2014) & PropAnemSamp >0][, c("year", "site")][, Monsoon := "NEM"], SurveyData[year %in% c(2012, 2013, 2014) & PropAnemSamp >0][, c("year", "site")][, Monsoon := "SWM"])  #what destinations were sampled, for use with unassigned table

###outside of the loop
PropToEval <- seq(0.1, 1, 0.1) #make a vector of proportions to sample iterativaley and compare
#empty table to hold results
PropSampOtherCAI <- data.table(TimeScale=character(), TimeID=character(), PropUnassigned=numeric(), ExpUnassigned=numeric(),  PropSampEval=numeric(), Check1=character(), Check2=character(), NrowSimConn=numeric())



In [59]:
pb <- txtProgressBar(min = 0, max =length(PropToEval), style = 3)

StartTime <- Sys.time()

for(i in 1:length(PropToEval)){

PropSampOtherCAI_int <- data.table(TimeScale=character(), TimeID=character(), PropUnassigned=numeric(), ExpUnassigned=numeric(),  PropSampEval=numeric(), Check1=character(), Check2=character(), NrowSimConn=numeric())[1:4]

dest_sampled <- date_join[DestPropSamp >0]
check1 <- nrow(dest_sampled)
dest_sampled <- dest_sampled[c(dest_sampled[, .I[Source != "Other"]], sample(dest_sampled[, .I[Source == "Other"]], length(dest_sampled[, .I[Source == "Other"]])*PropToEval[i]))]
check2 <- nrow(dest_sampled)
dest_sampled <- dest_sampled[c(dest_sampled[, .I[Source != "CAI"]], sample(dest_sampled[, .I[Source == "CAI"]], length(dest_sampled[, .I[Source == "CAI"]])*PropToEval[i]))]
check3 <- nrow(dest_sampled)

#check that we have less rows, should both be TRUE
test1 <- check1 > check2
test2 <- check2 > check3
#check1 > check2
#check2 > check3


#join in the number of parentage matches observed by year
dest_sampled <- kernels[Year %in% c("2012", "2013", "2014")][, Year:=as.integer(Year)][,c("Year", "NumParentageMatches")][dest_sampled, on=.(Year=YearSampled)]#[

#randomly subsample the sampled particle data
sim_sample <- dest_sampled[, .SD[sample(.N, DestNOffsAnnual, prob=SurvWeight)], by = c("Year", "Dest")] #randomly sample rows (particles) from the table according to the survival weighting, based on the number we sampled at each site in each year of surveys

PropUnassignedByYear <- (sim_sample[Source == "CAI"| Source == "Other", .(.N), by="Year"][, N]/#total particales sampled from other/CAI sources
sim_sample[, .(.N), by="Year"][, N]) #total particles sampled

PropSampOtherCAI_int$TimeScale[1] <- "annual"
PropSampOtherCAI_int$TimeID[1] <- "2012"
PropSampOtherCAI_int$PropUnassigned[1] <- PropUnassignedByYear[1]
PropSampOtherCAI_int$ExpUnassigned[1] <- 1-(ExpectedPropAssigned[1, ExpAssigned])
PropSampOtherCAI_int$PropSampEval[1] <- PropToEval[i]
PropSampOtherCAI_int$Check1[1] <- test1
PropSampOtherCAI_int$Check2[1] <- test2
PropSampOtherCAI_int$NrowSimConn[1] <- nrow(dest_sampled)
    
PropSampOtherCAI_int$TimeScale[2] <- "annual"
PropSampOtherCAI_int$TimeID[2] <- "2013"
PropSampOtherCAI_int$PropUnassigned[2] <- PropUnassignedByYear[2]
PropSampOtherCAI_int$ExpUnassigned[2] <- 1-(ExpectedPropAssigned[2, ExpAssigned])
PropSampOtherCAI_int$PropSampEval[2] <- PropToEval[i]
PropSampOtherCAI_int$Check1[2] <- test1
PropSampOtherCAI_int$Check2[2] <- test2
PropSampOtherCAI_int$NrowSimConn[2] <- nrow(dest_sampled)
    
PropSampOtherCAI_int$TimeScale[3] <- "annual"
PropSampOtherCAI_int$TimeID[3] <- "2014"
PropSampOtherCAI_int$PropUnassigned[3] <- PropUnassignedByYear[3]
PropSampOtherCAI_int$ExpUnassigned[3] <- 1-(ExpectedPropAssigned[3, ExpAssigned])
PropSampOtherCAI_int$PropSampEval[3] <- PropToEval[i]
PropSampOtherCAI_int$Check1[3] <- test1
PropSampOtherCAI_int$Check2[3] <- test2
PropSampOtherCAI_int$NrowSimConn[3] <- nrow(dest_sampled)
    
PropSampOtherCAI_int$TimeScale[4] <- "interannual"
PropSampOtherCAI_int$TimeID[4] <- "2012_4"
PropSampOtherCAI_int$PropUnassigned[4] <- nrow(sim_sample[Source == "CAI"| Source == "Other"])/nrow(sim_sample)
PropSampOtherCAI_int$ExpUnassigned[4] <- 1-(ExpectedPropAssigned[4, ExpAssigned])
PropSampOtherCAI_int$PropSampEval[4] <- PropToEval[i]
PropSampOtherCAI_int$Check1[4] <- test1
PropSampOtherCAI_int$Check2[4] <- test2
PropSampOtherCAI_int$NrowSimConn[4] <- nrow(dest_sampled)

l <- list(PropSampOtherCAI, PropSampOtherCAI_int)
PropSampOtherCAI <- rbindlist(l, use.names = TRUE, fill=TRUE, idcol = FALSE)
setTxtProgressBar(pb, i)
    
}

close(pb)
EndTime <- Sys.time()
EndTime-StartTime

fwrite(PropSampOtherCAI, file="~/oceanography/script_output/SimulationSummaryTables/PropSampOtherCAIEvaluation.csv")


  |======================================================================| 100%


Time difference of 1.066185 mins

In [58]:
PropSampOtherCAI[order(-PropUnassigned)]

TimeScale,TimeID,PropUnassigned,ExpUnassigned,PropSampEval,Check1,Check2,NrowSimConn
annual,2013,0.6666667,0.7583551,0.9,TRUE,TRUE,579512
annual,2013,0.6666667,0.7583551,1.0,FALSE,FALSE,610419
annual,2012,0.6507937,0.9274837,0.8,TRUE,TRUE,548606
annual,2014,0.6464088,0.8743125,0.9,TRUE,TRUE,579512
interannual,2012_4,0.6395939,0.8441326,0.9,TRUE,TRUE,579512
annual,2014,0.6353591,0.8743125,0.8,TRUE,TRUE,548606
interannual,2012_4,0.6294416,0.8441326,1.0,FALSE,FALSE,610419
interannual,2012_4,0.6269036,0.8441326,0.8,TRUE,TRUE,548606
annual,2014,0.6187845,0.8743125,1.0,FALSE,FALSE,610419
annual,2013,0.6066667,0.7583551,0.8,TRUE,TRUE,548606


__Seems like good justification to not subsample the ROMS particles from other/Camotes Islands__

[1] TRUE

[1] TRUE

[1] TRUE

In [10]:
nrow(dest_sampled)
nrow(SimConn)

[1] 363760

[1] 212861852

In [7]:
#save inter file
#fwrite(dest_sampled, file="~/oceanography/script_output/LongFormConnWithProbsTest.csv")
#see if I can write as a compressed file so it can be stored on github
#https://stackoverflow.com/questions/42788401/is-possible-to-use-fwrite-from-data-table-with-gzfile